# 62-前后端分离与API文档

## 📚 用途说明

**学习目标**：
- 掌握前后端分离架构的设计原理
- 学会实现RESTful API和标准化文档
- 理解跨域通信和安全策略
- 能够构建完整的前后端分离系统

**前置要求**：
- 已完成FastAPI和Web安全学习
- 熟练掌握HTTP协议和RESTful API
- 了解前端基础和JavaScript
- 理解API设计最佳实践

**与LangChain关联**：
- 前后端分离支持LangChain的AI服务部署
- API文档为LangChain集成提供清晰接口
- 跨域策略保障LangChain前端安全访问
- 为LangChain提供企业级API架构

---

## 🔢 知识点覆盖

### 7.7 前后端分离与API文档 [⭐⭐进阶]
**知识点说明**：前后端分离与API文档是现代Web应用的核心架构模式，为LangChain的企业级部署和前端集成提供重要的技术支撑。

**学习要求**：
- 掌握前后端分离架构的设计原理
- 学会实现RESTful API和标准化文档
- 理解跨域通信和安全策略
- 能够构建完整的前后端分离系统

**案例要求**：
- 实现完整的前后端分离架构
- 构建API文档和版本管理系统
- 开发跨域通信和安全机制
- 验证点：能独立设计前后端分离系统

In [ ]:
print("🌐 前后端分离与API文档:")
print("=" * 50)

# 导入必要的库
import asyncio
import time
import datetime
import uuid
import json
import random
import re
from typing import List, Dict, Any, Optional, Union, Type
from dataclasses import dataclass, asdict, field
from enum import Enum
from contextlib import asynccontextmanager
from pathlib import Path

# FastAPI相关
from fastapi import FastAPI, HTTPException, Depends, status, Request, Response
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse, HTMLResponse
from fastapi.openapi.utils import get_openapi
from pydantic import BaseModel, EmailStr, Field, validator
from pydantic.schema import schema

# API文档相关
from fastapi.openapi.docs import get_swagger_ui_html, get_redoc_html
from fastapi.staticfiles import StaticFiles

print(f"✅ Python版本: {__import__('sys').version}")
print(f"✅ 当前时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# 1. 前后端分离架构设计
print(f"📝 1. 前后端分离架构设计:")

# 1.1 分离架构核心组件
print(f"\n   🏗️ 1.1 分离架构核心组件:")

@dataclass
class SeparationConfig:
    """前后端分离配置"""
    # API配置
    api_prefix: str = "/api"
    api_version: str = "v1"
    api_title: str = "Python分离架构API"
    api_description: str = "前后端分离架构示例API"
    
    # 前端配置
    frontend_url: str = "http://localhost:3000"
    frontend_routes: List[str] = field(default_factory=lambda: ["/", "/login", "/dashboard"])
    
    # CORS配置
    cors_origins: List[str] = field(default_factory=lambda: ["http://localhost:3000", "http://127.0.0.1:3000"])
    cors_methods: List[str] = field(default_factory=lambda: ["GET", "POST", "PUT", "DELETE", "OPTIONS"])
    cors_headers: List[str] = field(default_factory=lambda: ["*"])
    cors_credentials: bool = True
    
    # API文档配置
    docs_url: str = "/docs"
    redoc_url: str = "/redoc"
    openapi_url: str = "/openapi.json"
    
    # 版本控制
    enable_versioning: bool = True
    default_version: str = "v1"
    supported_versions: List[str] = field(default_factory=lambda: ["v1", "v2"])

@dataclass
class APIResponse:
    """标准化API响应格式"""
    success: bool
    data: Any = None
    message: str = ""
    code: int = 200
    timestamp: datetime.datetime = field(default_factory=datetime.datetime.utcnow)
    request_id: str = field(default_factory=lambda: str(uuid.uuid4()))
    version: str = "v1"
    
    @classmethod
    def success_response(cls, data: Any = None, message: str = "操作成功", code: int = 200) -> "APIResponse":
        return cls(
            success=True,
            data=data,
            message=message,
            code=code
        )
    
    @classmethod
    def error_response(cls, message: str = "操作失败", code: int = 400, data: Any = None) -> "APIResponse":
        return cls(
            success=False,
            data=data,
            message=message,
            code=code
        )

class SeparationArchitecture:
    """前后端分离架构管理器"""
    
    def __init__(self, config: SeparationConfig):
        self.config = config
        self.app = None
        self.api_versions = {}
        
    def create_fastapi_app(self) -> FastAPI:
        """创建FastAPI应用"""
        # 创建应用
        self.app = FastAPI(
            title=self.config.api_title,
            description=self.config.api_description,
            version=self.config.api_version,
            docs_url=None,  # 禁用默认文档
            redoc_url=None,
            openapi_url=None
        )
        
        # 添加CORS中间件
        self.app.add_middleware(
            CORSMiddleware,
            allow_origins=self.config.cors_origins,
            allow_credentials=self.config.cors_credentials,
            allow_methods=self.config.cors_methods,
            allow_headers=self.config.cors_headers
        )
        
        # 添加全局异常处理器
        self._add_exception_handlers()
        
        # 添加中间件
        self.app.middleware("http")(self._add_request_context)
        
        return self.app
    
    def _add_exception_handlers(self):
        """添加异常处理器"""
        @self.app.exception_handler(HTTPException)
        async def http_exception_handler(request: Request, exc: HTTPException):
            return JSONResponse(
                status_code=exc.status_code,
                content=asdict(APIResponse.error_response(
                    message=exc.detail,
                    code=exc.status_code
                ))
            )
        
        @self.app.exception_handler(Exception)
        async def general_exception_handler(request: Request, exc: Exception):
            return JSONResponse(
                status_code=500,
                content=asdict(APIResponse.error_response(
                    message="服务器内部错误",
                    code=500
                ))
            )
    
    async def _add_request_context(self, request: Request, call_next):
        """添加请求上下文中间件"""
        start_time = time.time()
        
        # 添加请求ID
        request_id = str(uuid.uuid4())
        request.state.request_id = request_id
        
        # 处理请求
        response = await call_next(request)
        
        # 添加响应头
        response.headers["X-Request-ID"] = request_id
        response.headers["X-Response-Time"] = str(time.time() - start_time)
        response.headers["X-API-Version"] = self.config.api_version
        
        return response
    
    def setup_api_documentation(self):
        """设置API文档"""
        if not self.app:
            raise ValueError("请先创建FastAPI应用")
        
        # 自定义OpenAPI
        def custom_openapi():
            if self.app.openapi_schema:
                return self.app.openapi_schema
            
            openapi_schema = get_openapi(
                title=self.config.api_title,
                version=self.config.api_version,
                description=self.config.api_description,
                routes=self.app.routes,
            )
            
            # 添加自定义信息
            openapi_schema["info"]["x-logo"] = {
                "url": "https://fastapi.tiangolo.com/img/logo-margin/logo-teal.png"
            }
            
            # 添加安全定义
            openapi_schema["components"]["securitySchemes"] = {
                "BearerAuth": {
                    "type": "http",
                    "scheme": "bearer",
                    "bearerFormat": "JWT"
                }
            }
            
            self.app.openapi_schema = openapi_schema
            return self.app.openapi_schema
        
        self.app.openapi = custom_openapi
        
        # Swagger UI
        @self.app.get(self.config.docs_url, include_in_schema=False)
        async def custom_swagger_ui_html():
            return get_swagger_ui_html(
                openapi_url=self.config.openapi_url,
                title=self.config.api_title + " - Swagger UI",
                oauth2_redirect_url=self.app.swagger_ui_oauth2_redirect_url,
                swagger_js_url="https://cdn.jsdelivr.net/npm/swagger-ui-dist@5/swagger-ui-bundle.js",
                swagger_css_url="https://cdn.jsdelivr.net/npm/swagger-ui-dist@5/swagger-ui.css",
            )
        
        # ReDoc
        @self.app.get(self.config.redoc_url, include_in_schema=False)
        async def redoc_html():
            return get_redoc_html(
                openapi_url=self.config.openapi_url,
                title=self.config.api_title + " - ReDoc",
                redoc_js_url="https://cdn.jsdelivr.net/npm/redoc@2.0.0/bundles/redoc.standalone.js",
            )
        
        # OpenAPI JSON
        @self.app.get(self.config.openapi_url, include_in_schema=False)
        async def get_openapi_schema():
            return JSONResponse(self.app.openapi())
    
    def create_versioned_router(self, version: str) -> FastAPI:
        """创建版本化路由器"""
        if version not in self.config.supported_versions:
            raise ValueError(f"不支持的API版本: {version}")
        
        router = FastAPI(
            title=f"{self.config.api_title} - {version}",
            description=f"API版本 {version}",
            version=version
        )
        
        self.api_versions[version] = router
        return router
    
    def mount_versioned_routes(self):
        """挂载版本化路由"""
        if not self.app:
            raise ValueError("请先创建FastAPI应用")
        
        for version, router in self.api_versions.items():
            self.app.mount(f"{self.config.api_prefix}/{version}", router)
    
    def get_api_info(self) -> Dict[str, Any]:
        """获取API信息"""
        return {
            "title": self.config.api_title,
            "version": self.config.api_version,
            "description": self.config.api_description,
            "api_prefix": self.config.api_prefix,
            "supported_versions": self.config.supported_versions,
            "docs": {
                "swagger": f"{self.config.docs_url}",
                "redoc": f"{self.config.redoc_url}",
                "openapi": f"{self.config.openapi_url}"
            },
            "cors": {
                "origins": self.config.cors_origins,
                "methods": self.config.cors_methods,
                "credentials": self.config.cors_credentials
            }
        }

print(f"   ✅ 前后端分离架构设计完成")
print(f"      - SeparationConfig: 分离架构配置")
print(f"      - APIResponse: 标准化API响应格式")
print(f"      - SeparationArchitecture: 分离架构管理器")

print(f"\n✅ 前后端分离架构设计完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握前后端分离架构的设计原理")
print(f"   ✓ 学会实现RESTful API和标准化文档")
print(f"   ✓ 理解跨域通信和安全策略")
print(f"   ✓ 能够构建完整的前后端分离系统")

### API文档与标准化 [⭐⭐进阶]
**知识点说明**：API文档与标准化是前后端分离架构的重要组成部分，为LangChain的企业级集成和前端开发提供清晰的接口规范。

**学习要求**：
- 掌握OpenAPI/Swagger文档规范
- 学会实现自动化API文档生成
- 理解API版本控制和兼容性
- 能够设计标准化的API接口

**案例要求**：
- 实现完整的API文档系统
- 构建版本化API管理
- 开发标准化响应格式
- 验证点：能独立设计和维护API文档

In [ ]:
print("\n📚 API文档与标准化:")
print("=" * 50)

# 2. API文档与标准化
print(f"📝 2. API文档与标准化:")

# 2.1 数据模型和响应格式
print(f"\n   📋 2.1 数据模型和响应格式:")

# 用户相关模型
class UserBase(BaseModel):
    """用户基础模型"""
    username: str = Field(..., min_length=3, max_length=50, description="用户名")
    email: EmailStr = Field(..., description="邮箱地址")
    full_name: Optional[str] = Field(None, max_length=100, description="全名")
    
    @validator('username')
    def validate_username(cls, v):
        if not re.match(r'^[a-zA-Z0-9_]+$', v):
            raise ValueError('用户名只能包含字母、数字和下划线')
        return v

class UserCreate(UserBase):
    """用户创建模型"""
    password: str = Field(..., min_length=8, max_length=128, description="密码")
    
    @validator('password')
    def validate_password(cls, v):
        if not re.search(r'[A-Z]', v):
            raise ValueError('密码必须包含大写字母')
        if not re.search(r'[a-z]', v):
            raise ValueError('密码必须包含小写字母')
        if not re.search(r'\d', v):
            raise ValueError('密码必须包含数字')
        return v

class UserUpdate(BaseModel):
    """用户更新模型"""
    email: Optional[EmailStr] = Field(None, description="邮箱地址")
    full_name: Optional[str] = Field(None, max_length=100, description="全名")
    is_active: Optional[bool] = Field(None, description="是否激活")

class UserInDB(UserBase):
    """数据库中的用户模型"""
    id: int = Field(..., description="用户ID")
    hashed_password: str = Field(..., description="哈希密码")
    is_active: bool = Field(True, description="是否激活")
    created_at: datetime.datetime = Field(..., description="创建时间")
    updated_at: datetime.datetime = Field(..., description="更新时间")
    
    class Config:
        schema_extra = {
            "example": {
                "id": 1,
                "username": "john_doe",
                "email": "john@example.com",
                "full_name": "John Doe",
                "hashed_password": "hashed_password_here",
                "is_active": True,
                "created_at": "2023-01-01T00:00:00Z",
                "updated_at": "2023-01-01T00:00:00Z"
            }
        }

class UserResponse(UserBase):
    """用户响应模型"""
    id: int = Field(..., description="用户ID")
    is_active: bool = Field(..., description="是否激活")
    created_at: datetime.datetime = Field(..., description="创建时间")
    
    class Config:
        schema_extra = {
            "example": {
                "id": 1,
                "username": "john_doe",
                "email": "john@example.com",
                "full_name": "John Doe",
                "is_active": True,
                "created_at": "2023-01-01T00:00:00Z"
            }
        }

# 文章相关模型
class PostBase(BaseModel):
    """文章基础模型"""
    title: str = Field(..., min_length=1, max_length=200, description="标题")
    content: str = Field(..., min_length=1, description="内容")
    summary: Optional[str] = Field(None, max_length=500, description="摘要")
    tags: List[str] = Field(default_factory=list, description="标签")
    is_published: bool = Field(False, description="是否发布")

class PostCreate(PostBase):
    """文章创建模型"""
    pass

class PostUpdate(BaseModel):
    """文章更新模型"""
    title: Optional[str] = Field(None, min_length=1, max_length=200, description="标题")
    content: Optional[str] = Field(None, min_length=1, description="内容")
    summary: Optional[str] = Field(None, max_length=500, description="摘要")
    tags: Optional[List[str]] = Field(None, description="标签")
    is_published: Optional[bool] = Field(None, description="是否发布")

class PostResponse(PostBase):
    """文章响应模型"""
    id: int = Field(..., description="文章ID")
    author_id: int = Field(..., description="作者ID")
    author_name: str = Field(..., description="作者姓名")
    created_at: datetime.datetime = Field(..., description="创建时间")
    updated_at: datetime.datetime = Field(..., description="更新时间")
    view_count: int = Field(0, description="浏览次数")
    
    class Config:
        schema_extra = {
            "example": {
                "id": 1,
                "title": "Python异步编程",
                "content": "这是一篇关于Python异步编程的文章...",
                "summary": "Python异步编程入门指南",
                "tags": ["Python", "异步编程"],
                "is_published": True,
                "author_id": 1,
                "author_name": "John Doe",
                "created_at": "2023-01-01T00:00:00Z",
                "updated_at": "2023-01-01T00:00:00Z",
                "view_count": 100
            }
        }

# 分页模型
class PaginationParams(BaseModel):
    """分页参数"""
    page: int = Field(1, ge=1, description="页码")
    size: int = Field(10, ge=1, le=100, description="每页数量")
    
    @property
    def offset(self) -> int:
        return (self.page - 1) * self.size

class PaginatedResponse(BaseModel):
    """分页响应"""
    items: List[Any] = Field(..., description="数据列表")
    total: int = Field(..., description="总数量")
    page: int = Field(..., description="当前页码")
    size: int = Field(..., description="每页数量")
    pages: int = Field(..., description="总页数")
    
    @classmethod
    def create(cls, items: List[Any], total: int, pagination: PaginationParams) -> "PaginatedResponse":
        pages = (total + pagination.size - 1) // pagination.size
        return cls(
            items=items,
            total=total,
            page=pagination.page,
            size=pagination.size,
            pages=pages
        )

# 2.2 API文档生成器
print(f"\n   📖 2.2 API文档生成器:")

class APIDocumentationGenerator:
    """API文档生成器"""
    
    def __init__(self, app: FastAPI):
        self.app = app
        self.custom_schemas = {}
        self.tag_descriptions = {}
    
    def add_tag_description(self, name: str, description: str):
        """添加标签描述"""
        self.tag_descriptions[name] = description
    
    def add_custom_schema(self, name: str, schema_dict: Dict[str, Any]):
        """添加自定义模式"""
        self.custom_schemas[name] = schema_dict
    
    def generate_openapi_spec(self) -> Dict[str, Any]:
        """生成OpenAPI规范"""
        openapi_schema = get_openapi(
            title="Python前后端分离API",
            version="1.0.0",
            description="完整的前后端分离API示例",
            routes=self.app.routes,
        )
        
        # 添加自定义信息
        openapi_schema["info"].update({
            "contact": {
                "name": "API Support",
                "email": "support@example.com"
            },
            "license": {
                "name": "MIT",
                "url": "https://opensource.org/licenses/MIT"
            }
        })
        
        # 添加服务器信息
        openapi_schema["servers"] = [
            {
                "url": "http://localhost:8000",
                "description": "开发服务器"
            },
            {
                "url": "https://api.example.com",
                "description": "生产服务器"
            }
        ]
        
        # 添加标签描述
        openapi_schema["tags"] = [
            {
                "name": name,
                "description": description
            }
            for name, description in self.tag_descriptions.items()
        ]
        
        # 添加安全定义
        openapi_schema["components"]["securitySchemes"] = {
            "BearerAuth": {
                "type": "http",
                "scheme": "bearer",
                "bearerFormat": "JWT",
                "description": "JWT认证令牌"
            },
            "ApiKeyAuth": {
                "type": "apiKey",
                "in": "header",
                "name": "X-API-Key",
                "description": "API密钥认证"
            }
        }
        
        # 添加自定义模式
        if self.custom_schemas:
            openapi_schema["components"]["schemas"].update(self.custom_schemas)
        
        return openapi_schema
    
    def export_documentation(self, format: str = "json", filepath: str = None) -> str:
        """导出文档"""
        schema = self.generate_openapi_spec()
        
        if format == "json":
            content = json.dumps(schema, indent=2, ensure_ascii=False)
            extension = ".json"
        elif format == "yaml":
            try:
                import yaml
                content = yaml.dump(schema, default_flow_style=False, allow_unicode=True)
                extension = ".yaml"
            except ImportError:
                raise ImportError("需要安装PyYAML: pip install PyYAML")
        else:
            raise ValueError("不支持的格式，请使用 'json' 或 'yaml'")
        
        if filepath:
            if not filepath.endswith(extension):
                filepath += extension
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(content)
            return filepath
        
        return content
    
    def generate_client_code(self, language: str = "javascript") -> str:
        """生成客户端代码"""
        schema = self.generate_openapi_spec()
        
        if language == "javascript":
            return self._generate_javascript_client(schema)
        elif language == "python":
            return self._generate_python_client(schema)
        else:
            raise ValueError(f"不支持的语言: {language}")
    
    def _generate_javascript_client(self, schema: Dict[str, Any]) -> str:
        """生成JavaScript客户端代码"""
        client_code = '''
// API客户端代码 (JavaScript)
class APIClient {
    constructor(baseURL = 'http://localhost:8000/api/v1') {
        this.baseURL = baseURL;
        this.token = null;
    }
    
    setToken(token) {
        this.token = token;
    }
    
    async request(method, endpoint, data = null, params = {}) {
        const url = new URL(endpoint, this.baseURL);
        Object.keys(params).forEach(key => 
            url.searchParams.append(key, params[key])
        );
        
        const config = {
            method,
            headers: {
                'Content-Type': 'application/json',
            }
        };
        
        if (this.token) {
            config.headers.Authorization = `Bearer ${this.token}`;
        }
        
        if (data) {
            config.body = JSON.stringify(data);
        }
        
        const response = await fetch(url, config);
        const result = await response.json();
        
        if (!response.ok) {
            throw new Error(result.message || '请求失败');
        }
        
        return result;
    }
    
    // 用户相关API
    async getUsers(params = {}) {
        return this.request('GET', '/users', null, params);
    }
    
    async getUser(userId) {
        return this.request('GET', `/users/${userId}`);
    }
    
    async createUser(userData) {
        return this.request('POST', '/users', userData);
    }
    
    async updateUser(userId, userData) {
        return this.request('PUT', `/users/${userId}`, userData);
    }
    
    async deleteUser(userId) {
        return this.request('DELETE', `/users/${userId}`);
    }
    
    // 文章相关API
    async getPosts(params = {}) {
        return this.request('GET', '/posts', null, params);
    }
    
    async getPost(postId) {
        return this.request('GET', `/posts/${postId}`);
    }
    
    async createPost(postData) {
        return this.request('POST', '/posts', postData);
    }
    
    async updatePost(postId, postData) {
        return this.request('PUT', `/posts/${postId}`, postData);
    }
    
    async deletePost(postId) {
        return this.request('DELETE', `/posts/${postId}`);
    }
}

// 使用示例
const api = new APIClient();
api.setToken('your-jwt-token-here');
        '''
        return client_code.strip()
    
    def _generate_python_client(self, schema: Dict[str, Any]) -> str:
        """生成Python客户端代码"""
        client_code = '''
# API客户端代码 (Python)
import requests
from typing import Dict, Any, Optional, List

class APIClient:
    def __init__(self, base_url: str = "http://localhost:8000/api/v1"):
        self.base_url = base_url
        self.token: Optional[str] = None
        self.session = requests.Session()
    
    def set_token(self, token: str):
        """设置认证令牌"""
        self.token = token
        self.session.headers.update({
            "Authorization": f"Bearer {token}"
        })
    
    def request(self, method: str, endpoint: str, 
                data: Optional[Dict[str, Any]] = None, 
                params: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        """发送HTTP请求"""
        url = f"{self.base_url}{endpoint}"
        
        response = self.session.request(
            method=method,
            url=url,
            json=data,
            params=params
        )
        
        response.raise_for_status()
        return response.json()
    
    # 用户相关API
    def get_users(self, params: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        """获取用户列表"""
        return self.request("GET", "/users", params=params)
    
    def get_user(self, user_id: int) -> Dict[str, Any]:
        """获取单个用户"""
        return self.request("GET", f"/users/{user_id}")
    
    def create_user(self, user_data: Dict[str, Any]) -> Dict[str, Any]:
        """创建用户"""
        return self.request("POST", "/users", data=user_data)
    
    def update_user(self, user_id: int, user_data: Dict[str, Any]) -> Dict[str, Any]:
        """更新用户"""
        return self.request("PUT", f"/users/{user_id}", data=user_data)
    
    def delete_user(self, user_id: int) -> Dict[str, Any]:
        """删除用户"""
        return self.request("DELETE", f"/users/{user_id}")
    
    # 文章相关API
    def get_posts(self, params: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        """获取文章列表"""
        return self.request("GET", "/posts", params=params)
    
    def get_post(self, post_id: int) -> Dict[str, Any]:
        """获取单个文章"""
        return self.request("GET", f"/posts/{post_id}")
    
    def create_post(self, post_data: Dict[str, Any]) -> Dict[str, Any]:
        """创建文章"""
        return self.request("POST", "/posts", data=post_data)
    
    def update_post(self, post_id: int, post_data: Dict[str, Any]) -> Dict[str, Any]:
        """更新文章"""
        return self.request("PUT", f"/posts/{post_id}", data=post_data)
    
    def delete_post(self, post_id: int) -> Dict[str, Any]:
        """删除文章"""
        return self.request("DELETE", f"/posts/{post_id}")

# 使用示例
api = APIClient()
api.set_token("your-jwt-token-here")
users = api.get_users()
print(users)
        '''
        return client_code.strip()

# 2.3 版本控制管理
print(f"\n   🔄 2.3 版本控制管理:")

class APIVersionManager:
    """API版本管理器"""
    
    def __init__(self):
        self.versions = {}
        self.deprecation_policies = {}
        self.migration_guides = {}
    
    def register_version(self, version: str, description: str, 
                         deprecation_date: Optional[datetime.datetime] = None):
        """注册API版本"""
        self.versions[version] = {
            "description": description,
            "created_at": datetime.datetime.utcnow(),
            "deprecation_date": deprecation_date,
            "is_deprecated": deprecation_date and deprecation_date < datetime.datetime.utcnow()
        }
    
    def set_deprecation_policy(self, version: str, policy: Dict[str, Any]):
        """设置弃用策略"""
        self.deprecation_policies[version] = policy
    
    def add_migration_guide(self, from_version: str, to_version: str, guide: str):
        """添加迁移指南"""
        key = f"{from_version}->{to_version}"
        self.migration_guides[key] = guide
    
    def get_version_info(self, version: str) -> Optional[Dict[str, Any]]:
        """获取版本信息"""
        return self.versions.get(version)
    
    def list_versions(self) -> List[Dict[str, Any]]:
        """列出所有版本"""
        return [
            {
                "version": version,
                **info
            }
            for version, info in self.versions.items()
        ]
    
    def get_active_versions(self) -> List[str]:
        """获取活跃版本"""
        return [
            version for version, info in self.versions.items()
            if not info["is_deprecated"]
        ]
    
    def check_version_compatibility(self, client_version: str, 
                                    server_version: str) -> Dict[str, Any]:
        """检查版本兼容性"""
        client_info = self.versions.get(client_version)
        server_info = self.versions.get(server_version)
        
        if not client_info or not server_info:
            return {"compatible": False, "reason": "版本不存在"}
        
        if client_info["is_deprecated"]:
            return {
                "compatible": True,
                "warning": f"客户端版本 {client_version} 已弃用",
                "migration_required": True
            }
        
        # 简单的版本比较逻辑
        if self._compare_versions(client_version, server_version) < 0:
            return {
                "compatible": True,
                "warning": f"建议升级到版本 {server_version}",
                "migration_recommended": True
            }
        
        return {"compatible": True}
    
    def _compare_versions(self, v1: str, v2: str) -> int:
        """比较版本号"""
        def version_key(v):
            return [int(x) for x in v.replace('v', '').split('.')]
        
        v1_parts = version_key(v1)
        v2_parts = version_key(v2)
        
        for a, b in zip(v1_parts, v2_parts):
            if a < b:
                return -1
            elif a > b:
                return 1
        
        return 0

print(f"   ✅ API文档与标准化完成")
print(f"      - UserBase/UserCreate/UserUpdate: 用户数据模型")
print(f"      - PostBase/PostCreate/PostUpdate: 文章数据模型")
print(f"      - APIDocumentationGenerator: API文档生成器")
print(f"      - APIVersionManager: API版本管理器")

print(f"\n✅ API文档与标准化完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握OpenAPI/Swagger文档规范")
print(f"   ✓ 学会实现自动化API文档生成")
print(f"   ✓ 理解API版本控制和兼容性")
print(f"   ✓ 能够设计标准化的API接口")

### 前端集成与跨域通信 [⭐⭐进阶]
**知识点说明**：前端集成与跨域通信是前后端分离架构的关键技术，为LangChain的前端应用和AI服务集成提供完整的通信解决方案。

**学习要求**：
- 掌握CORS跨域通信机制
- 学会实现前端API客户端
- 理解认证和授权流程
- 能够构建完整的前后端通信系统

**案例要求**：
- 实现完整的前端API客户端
- 构建跨域通信和安全机制
- 开发认证和授权流程
- 验证点：能独立开发前后端通信系统

In [ ]:
print("\n🔗 前端集成与跨域通信:")
print("=" * 50)

# 3. 前端集成与跨域通信
print(f"📝 3. 前端集成与跨域通信:")

# 3.1 前端API客户端
print(f"\n   💻 3.1 前端API客户端:")

class FrontendAPIClient:
    """前端API客户端"""
    
    def __init__(self, base_url: str, api_version: str = "v1"):
        self.base_url = base_url.rstrip('/')
        self.api_version = api_version
        self.api_endpoint = f"{self.base_url}/api/{api_version}"
        self.token = None
        self.refresh_token = None
        
        # 请求拦截器
        self.request_interceptors = []
        self.response_interceptors = []
        
        # 错误处理
        self.error_handlers = {
            401: self._handle_auth_error,
            403: self._handle_forbidden_error,
            404: self._handle_not_found_error,
            429: self._handle_rate_limit_error,
            500: self._handle_server_error
        }
    
    def set_auth_tokens(self, access_token: str, refresh_token: str = None):
        """设置认证令牌"""
        self.token = access_token
        self.refresh_token = refresh_token
    
    def add_request_interceptor(self, interceptor):
        """添加请求拦截器"""
        self.request_interceptors.append(interceptor)
    
    def add_response_interceptor(self, interceptor):
        """添加响应拦截器"""
        self.response_interceptors.append(interceptor)
    
    async def request(self, method: str, endpoint: str, 
                     data: Dict[str, Any] = None, 
                     params: Dict[str, Any] = None,
                     headers: Dict[str, str] = None,
                     files: Dict[str, Any] = None) -> Dict[str, Any]:
        """发送HTTP请求"""
        import aiohttp
        
        url = f"{self.api_endpoint}{endpoint}"
        
        # 准备请求头
        request_headers = {
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        
        if self.token:
            request_headers["Authorization"] = f"Bearer {self.token}"
        
        if headers:
            request_headers.update(headers)
        
        # 应用请求拦截器
        request_data = {
            "method": method,
            "url": url,
            "headers": request_headers,
            "params": params,
            "data": data,
            "files": files
        }
        
        for interceptor in self.request_interceptors:
            request_data = interceptor(request_data)
        
        # 发送请求
        async with aiohttp.ClientSession() as session:
            try:
                if files:
                    # 文件上传
                    form_data = aiohttp.FormData()
                    for key, value in files.items():
                        form_data.add_field(key, value)
                    
                    async with session.request(
                        method=request_data["method"],
                        url=request_data["url"],
                        headers=request_data["headers"],
                        data=form_data,
                        params=request_data["params"]
                    ) as response:
                        response_data = await response.json()
                        status_code = response.status
                else:
                    # 普通请求
                    async with session.request(
                        method=request_data["method"],
                        url=request_data["url"],
                        headers=request_data["headers"],
                        json=request_data["data"],
                        params=request_data["params"]
                    ) as response:
                        response_data = await response.json()
                        status_code = response.status
                
                # 应用响应拦截器
                response_context = {
                    "data": response_data,
                    "status_code": status_code,
                    "headers": dict(response.headers)
                }
                
                for interceptor in self.response_interceptors:
                    response_context = interceptor(response_context)
                
                # 错误处理
                if status_code >= 400:
                    error_handler = self.error_handlers.get(status_code)
                    if error_handler:
                        await error_handler(response_context)
                
                return response_context["data"]
            
            except aiohttp.ClientError as e:
                raise Exception(f"网络请求失败: {str(e)}")
            except Exception as e:
                raise Exception(f"请求处理失败: {str(e)}")
    
    async def get(self, endpoint: str, params: Dict[str, Any] = None) -> Dict[str, Any]:
        """GET请求"""
        return await self.request("GET", endpoint, params=params)
    
    async def post(self, endpoint: str, data: Dict[str, Any] = None) -> Dict[str, Any]:
        """POST请求"""
        return await self.request("POST", endpoint, data=data)
    
    async def put(self, endpoint: str, data: Dict[str, Any] = None) -> Dict[str, Any]:
        """PUT请求"""
        return await self.request("PUT", endpoint, data=data)
    
    async def delete(self, endpoint: str) -> Dict[str, Any]:
        """DELETE请求"""
        return await self.request("DELETE", endpoint)
    
    async def upload(self, endpoint: str, files: Dict[str, Any], 
                     data: Dict[str, Any] = None) -> Dict[str, Any]:
        """文件上传"""
        return await self.request("POST", endpoint, data=data, files=files)
    
    # 错误处理方法
    async def _handle_auth_error(self, response_context: Dict[str, Any]):
        """处理认证错误"""
        if self.refresh_token:
            # 尝试刷新令牌
            try:
                refresh_response = await self.request(
                    "POST", "/auth/refresh", 
                    data={"refresh_token": self.refresh_token}
                )
                if refresh_response.get("success"):
                    self.token = refresh_response["data"]["access_token"]
                    return  # 重新发送原始请求
            except Exception:
                pass
        
        # 清除令牌并跳转登录
        self.token = None
        self.refresh_token = None
        raise Exception("认证失败，请重新登录")
    
    async def _handle_forbidden_error(self, response_context: Dict[str, Any]):
        """处理权限错误"""
        raise Exception("权限不足，无法访问此资源")
    
    async def _handle_not_found_error(self, response_context: Dict[str, Any]):
        """处理404错误"""
        raise Exception("请求的资源不存在")
    
    async def _handle_rate_limit_error(self, response_context: Dict[str, Any]):
        """处理限流错误"""
        raise Exception("请求过于频繁，请稍后重试")
    
    async def _handle_server_error(self, response_context: Dict[str, Any]):
        """处理服务器错误"""
        raise Exception("服务器内部错误，请稍后重试")

# 3.2 CORS配置管理
print(f"\n   🌐 3.2 CORS配置管理:")

@dataclass
class CORSConfiguration:
    """CORS配置"""
    # 允许的源
    allow_origins: List[str] = field(default_factory=list)
    allow_origin_regex: Optional[str] = None
    
    # 允许的方法
    allow_methods: List[str] = field(default_factory=lambda: ["GET"])
    
    # 允许的头
    allow_headers: List[str] = field(default_factory=list)
    
    # 暴露的头
    expose_headers: List[str] = field(default_factory=list)
    
    # 凭证支持
    allow_credentials: bool = False
    
    # 预检缓存
    max_age: int = 600
    
    # 环境配置
    environment: str = "development"  # development, staging, production

class CORSManager:
    """CORS管理器"""
    
    def __init__(self, config: CORSConfiguration):
        self.config = config
        self._setup_environment_defaults()
    
    def _setup_environment_defaults(self):
        """设置环境默认值"""
        if self.config.environment == "development":
            if not self.config.allow_origins:
                self.config.allow_origins = [
                    "http://localhost:3000",
                    "http://localhost:8080",
                    "http://127.0.0.1:3000",
                    "http://127.0.0.1:8080"
                ]
            if not self.config.allow_methods:
                self.config.allow_methods = ["GET", "POST", "PUT", "DELETE", "OPTIONS"]
            if not self.config.allow_headers:
                self.config.allow_headers = ["*"]
            self.config.allow_credentials = True
        
        elif self.config.environment == "production":
            if not self.config.allow_origins:
                self.config.allow_origins = ["https://yourdomain.com"]
            if not self.config.allow_methods:
                self.config.allow_methods = ["GET", "POST", "PUT", "DELETE"]
            if not self.config.allow_headers:
                self.config.allow_headers = [
                    "Content-Type", "Authorization", "X-Requested-With"
                ]
            self.config.max_age = 86400  # 24小时
    
    def create_cors_middleware(self) -> CORSMiddleware:
        """创建CORS中间件"""
        return CORSMiddleware(
            allow_origins=self.config.allow_origins,
            allow_origin_regex=self.config.allow_origin_regex,
            allow_methods=self.config.allow_methods,
            allow_headers=self.config.allow_headers,
            expose_headers=self.config.expose_headers,
            allow_credentials=self.config.allow_credentials,
            max_age=self.config.max_age
        )
    
    def validate_origin(self, origin: str) -> bool:
        """验证源是否被允许"""
        if not origin:
            return False
        
        # 检查精确匹配
        if origin in self.config.allow_origins:
            return True
        
        # 检查正则表达式
        if self.config.allow_origin_regex:
            import re
            if re.match(self.config.allow_origin_regex, origin):
                return True
        
        # 检查通配符
        if "*" in self.config.allow_origins:
            return True
        
        return False
    
    def get_cors_info(self) -> Dict[str, Any]:
        """获取CORS配置信息"""
        return {
            "environment": self.config.environment,
            "allow_origins": self.config.allow_origins,
            "allow_methods": self.config.allow_methods,
            "allow_headers": self.config.allow_headers,
            "allow_credentials": self.config.allow_credentials,
            "max_age": self.config.max_age
        }

# 3.3 前端集成示例
print(f"\n   🎨 3.3 前端集成示例:")

def generate_frontend_integration_example() -> str:
    """生成前端集成示例代码"""
    return '''
// 前端集成示例 (React + TypeScript)

// 1. API客户端配置
import axios, { AxiosInstance, AxiosRequestConfig, AxiosResponse } from 'axios';

class APIClient {
    private client: AxiosInstance;
    private refreshToken: string | null = null;
    
    constructor(baseURL: string) {
        this.client = axios.create({
            baseURL: `${baseURL}/api/v1`,
            timeout: 10000,
            headers: {
                'Content-Type': 'application/json',
            },
        });
        
        this.setupInterceptors();
    }
    
    private setupInterceptors() {
        // 请求拦截器
        this.client.interceptors.request.use(
            (config) => {
                const token = localStorage.getItem('access_token');
                if (token) {
                    config.headers.Authorization = `Bearer ${token}`;
                }
                return config;
            },
            (error) => Promise.reject(error)
        );
        
        // 响应拦截器
        this.client.interceptors.response.use(
            (response) => response.data,
            async (error) => {
                const originalRequest = error.config;
                
                if (error.response?.status === 401 && !originalRequest._retry) {
                    originalRequest._retry = true;
                    
                    try {
                        const refreshToken = localStorage.getItem('refresh_token');
                        if (refreshToken) {
                            const response = await this.client.post('/auth/refresh', {
                                refresh_token: refreshToken
                            });
                            
                            const { access_token } = response.data;
                            localStorage.setItem('access_token', access_token);
                            
                            return this.client(originalRequest);
                        }
                    } catch (refreshError) {
                        localStorage.removeItem('access_token');
                        localStorage.removeItem('refresh_token');
                        window.location.href = '/login';
                    }
                }
                
                return Promise.reject(error);
            }
        );
    }
    
    // 用户相关API
    async getUsers(params?: any) {
        return this.client.get('/users', { params });
    }
    
    async getUser(id: number) {
        return this.client.get(`/users/${id}`);
    }
    
    async createUser(data: any) {
        return this.client.post('/users', data);
    }
    
    async updateUser(id: number, data: any) {
        return this.client.put(`/users/${id}`, data);
    }
    
    async deleteUser(id: number) {
        return this.client.delete(`/users/${id}`);
    }
    
    // 认证相关API
    async login(credentials: { username: string; password: string }) {
        const response = await this.client.post('/auth/login', credentials);
        const { access_token, refresh_token } = response.data;
        
        localStorage.setItem('access_token', access_token);
        if (refresh_token) {
            localStorage.setItem('refresh_token', refresh_token);
        }
        
        return response;
    }
    
    async logout() {
        try {
            await this.client.post('/auth/logout');
        } finally {
            localStorage.removeItem('access_token');
            localStorage.removeItem('refresh_token');
        }
    }
}

// 2. React Hook示例
import { useState, useEffect } from 'react';

export const useUsers = () => {
    const [users, setUsers] = useState([]);
    const [loading, setLoading] = useState(false);
    const [error, setError] = useState<string | null>(null);
    
    const apiClient = new APIClient('http://localhost:8000');
    
    const fetchUsers = async (params?: any) => {
        setLoading(true);
        setError(null);
        
        try {
            const response = await apiClient.getUsers(params);
            setUsers(response.data.items);
        } catch (err) {
            setError(err.message || '获取用户列表失败');
        } finally {
            setLoading(false);
        }
    };
    
    const createUser = async (userData: any) => {
        try {
            const response = await apiClient.createUser(userData);
            setUsers(prev => [...prev, response.data]);
            return response.data;
        } catch (err) {
            setError(err.message || '创建用户失败');
            throw err;
        }
    };
    
    useEffect(() => {
        fetchUsers();
    }, []);
    
    return {
        users,
        loading,
        error,
        fetchUsers,
        createUser
    };
};

// 3. 组件示例
import React from 'react';

const UserList: React.FC = () => {
    const { users, loading, error, fetchUsers, createUser } = useUsers();
    
    const handleCreateUser = async () => {
        try {
            await createUser({
                username: 'newuser',
                email: 'newuser@example.com',
                password: 'password123',
                full_name: 'New User'
            });
        } catch (err) {
            console.error('创建用户失败:', err);
        }
    };
    
    if (loading) return <div>加载中...</div>;
    if (error) return <div>错误: {error}</div>;
    
    return (
        <div>
            <h2>用户列表</h2>
            <button onClick={handleCreateUser}>创建用户</button>
            <ul>
                {users.map((user: any) => (
                    <li key={user.id}>
                        {user.username} - {user.email}
                    </li>
                ))}
            </ul>
        </div>
    );
};

export default UserList;
    '''

# 创建前端集成示例
frontend_example = generate_frontend_integration_example()

print(f"   ✅ 前端集成与跨域通信完成")
print(f"      - FrontendAPIClient: 前端API客户端")
print(f"      - CORSConfiguration: CORS配置")
print(f"      - CORSManager: CORS管理器")
print(f"      - 前端集成示例: React + TypeScript")

print(f"\n✅ 前端集成与跨域通信完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握CORS跨域通信机制")
print(f"   ✓ 学会实现前端API客户端")
print(f"   ✓ 理解认证和授权流程")
print(f"   ✓ 能够构建完整的前后端通信系统")

## 📋 学习总结

### 知识清单达成情况

#### ✅ 已完成知识点

**7.7 前后端分离与API文档 [⭐⭐进阶]**
- ✅ 掌握前后端分离架构的设计原理
- ✅ 学会实现RESTful API和标准化文档
- ✅ 理解跨域通信和安全策略
- ✅ 能够构建完整的前后端分离系统

#### 🎯 核心技能掌握

**分离架构设计**
- ✅ 前后端分离架构原理和组件
- ✅ 标准化API响应格式设计
- ✅ API版本控制和兼容性管理
- ✅ 全局异常处理和中间件

**API文档标准化**
- ✅ OpenAPI/Swagger文档规范
- ✅ 自动化API文档生成
- ✅ Pydantic数据模型设计
- ✅ 客户端代码自动生成

**前端集成通信**
- ✅ CORS跨域通信机制
- ✅ 前端API客户端设计
- ✅ 认证令牌管理和刷新
- ✅ 请求拦截器和错误处理

#### 🚀 实践应用能力

**架构实现**
- ✅ 完整的前后端分离架构
- ✅ 版本化API管理系统
- ✅ 标准化响应格式设计
- ✅ 安全通信和认证机制

**文档管理**
- ✅ 自动化API文档生成
- ✅ 多格式文档导出
- ✅ 客户端SDK生成
- ✅ API版本迁移指南

#### 📊 与LangChain的关联

**企业级集成**
- ✅ 支持LangChain的AI服务前端集成
- ✅ 为LangChain提供标准化API接口
- ✅ 保障LangChain前后端安全通信
- ✅ 满足LangChain企业级部署需求

**开发效率**
- ✅ 自动化文档提升开发效率
- ✅ 标准化接口降低集成成本
- ✅ 版本控制保障系统稳定性
- ✅ 客户端生成简化前端开发

---

## 🎯 学习成果

通过本节学习，你已经掌握了：

1. **前后端分离架构** - 理解分离架构设计原理，掌握标准化API设计和版本控制
2. **API文档标准化** - 实现OpenAPI文档生成，掌握自动化文档管理和客户端代码生成
3. **前端集成通信** - 构建完整的前端API客户端，掌握CORS跨域和认证流程
4. **企业级解决方案** - 设计符合生产环境要求的完整前后端分离系统

这些技能为后续的微服务架构、Web应用部署等高级主题奠定了坚实基础，也为LangChain的企业级前端集成和API服务提供了重要的技术支撑。